In [ ]:
%matplotlib inline
import sys, os
import numpy as np
import inspect
from openxrd import find_peaks_2d
from openxrd import find_peaks_1d
from openxrd import fit_single
from openxrd import get_fit_all_1d
from openxrd import get_fit_all_2d
from openxrd import fits_to_csv_multitype
from openxrd import fit_data_to_csv
from openxrd import BrukerData
from openxrd import fit_multipeak
from openxrd import fits_to_csv
from openxrd import csv_append_col

from datasetmetta import get_name_data

import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.patheffects as path_effects
from mpl_toolkits.axes_grid1 import make_axes_locatable

import csv
from lmfit import models
from lmfit import lineshapes


def merge_data(files):
    # Merge files into 1 for analysis
    data_list = []
    file_list = []
    for i, f in enumerate(sorted(files)):
        datafile = str(f)
        # Maybe add output that says joined bla
        # print(os.path.basename(datafile)[:-11])
        if len(data_list) < 1:
            data_list.append(BrukerData(datafile))
        else:
            data = BrukerData(datafile)
            diff = data.x[0] - data_list[-1].x[-1]
            step = data.rngs[0].metta['step_size']
            if (np.array_equal(data_list[-1].y, data.y) and
                    step*-2 < diff < 2*step):  # are ys same and xs overlap?
                data_list[-1] = data_list[-1] + data
            else:
                data_list.append(data)
        file_list.append(datafile)
    return data_list, file_list


def get_peaks(peak_file, ax, x, y):
        # fastest?
        # https://softwarerecs.stackexchange.com/questions/7463/fastest-python-library-to-read-a-csv-file
        with open(peak_file, 'r') as f:
            peaks = csv.reader(row for row in f if not
                               row.startswith('#'))
            for peak in peaks:
                if (x.min() < float(peak[0]) < x.max()):
                    amp = y.max() * float(peak[1]) / 100.0
                    txt = ax.text(float(peak[0]), amp, peak[2],
                                  fontdict=labels, rotation=90)
                    txt.set_path_effects([path_effects.Stroke(linewidth=1,
                                         foreground='black'),
                                         path_effects.Normal()])

# Get data
root = '/mnt/W/Austin_Fox/'
rootf = root + 'XRD/'
#files = [rootf + '5582_map.raw']
files = [rootf + '8723_map.raw']
#rootf = 'exfiles/'
#files = [rootf + 'insit.raw']
#files = [rootf + 'Map-20.raw', rootf + 'Map-50.raw', rootf + 'Map-80.raw']
data_list, file_list = merge_data(files)
print(len(data_list))

#PLOT Setup

axlabels = {'family':               'sans',
            'color':                'black',
            'weight':               'normal',
            'size':                 20,
            }
titles = {'family':                 'sans',
          'color':                  'black',
          'weight':                 'normal',
          'size':                   24,
          }
labels = {'family':                 'sans',
          'fontname':               'DejaVu Sans',
          'color':                  '#66ff33',
          'weight':                 'normal',
          'size':                   14,
          'verticalalignment':      'center',
          'horizontalalignment':    'right'
          }

fig = plt.figure(figsize=(6, 6), dpi=100)
ax = fig.add_subplot(111)
ax.set_title('test', fontdict=titles)
#axis Lables
ax.set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
ax.set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
# Set axis tick labels font
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    for prop in axlabels:
        getattr(label, 'set_' + prop)(axlabels[prop])


for i, data in enumerate(data_list):
    for j, rng in enumerate(data.rngs):
        th_0 = rng.metta['start_theta']
        twoth_0 = rng.metta['start_2th']
        
        step_size = rng.metta['step_size']
        steps = rng.metta['steps']

        length = step_size*steps
        end = twoth_0 + length
        
        x = np.linspace(twoth_0, end, steps)
        y = np.asarray(rng.counts_data)
        #print(rng.metta['drive_x'], rng.metta['drive_y'],
        #      rng.metta['drive_phi'])
        #print(len(x), len(y))
        ax.plot(x,y, 'C'+str(j), linewidth=1, label=str(j+1)) #plot
        out = fit_single(x, y)
        ax.plot(x, out.best_fit,  'C'+str(j), linewidth=1,)
        print(j+1, 'fwhm:', out.report['fwhm'])
        print(j+1, 'center:', out.report['center'])

        #for key in out.report:
        #    print(key, out.report[key])
        
plt.tight_layout()
plt.show()


